In [1]:
import os
import shutil

# 정리할 폴더 경로
folder_paths = [
    'C:\\Users\\ohbok\\Taipy\\sparc-malfunction-classification-develop\\src\\.taipy',
    'C:\\Users\\ohbok\\Taipy\\sparc-malfunction-classification-develop\\src\\user_data'
]

for path in folder_paths:
    # 폴더가 존재하는지 확인
    if os.path.exists(path) and os.path.isdir(path):
        try:
            # 폴더 하위의 모든 항목을 순회하며 삭제
            for item in os.listdir(path):
                item_path = os.path.join(path, item)
                if os.path.isdir(item_path):
                    # 하위 폴더인 경우 shutil.rmtree로 삭제
                    shutil.rmtree(item_path)
                else:
                    # 파일인 경우 os.remove로 삭제
                    os.remove(item_path)
            print(f"폴더 '{path}'의 하위 내용이 성공적으로 삭제되었습니다.")
        except OSError as e:
            print(f"폴더 '{path}'의 하위 내용 삭제 중 오류가 발생했습니다: {e}")
    else:
        print(f"폴더 '{path}'가 존재하지 않거나 디렉토리가 아닙니다.")

폴더 'C:\Users\ohbok\Taipy\sparc-malfunction-classification-develop\src\.taipy'의 하위 내용이 성공적으로 삭제되었습니다.
폴더 'C:\Users\ohbok\Taipy\sparc-malfunction-classification-develop\src\user_data'의 하위 내용이 성공적으로 삭제되었습니다.


In [2]:
# 데이터 처리 및 기타 작업을 위한 라이브러리들을 불러옵니다.
import pandas as pd                                         # 데이터프레임 구조를 다루는 데 필수적인 라이브러리
import numpy as np                                          # 숫자 연산을 위한 라이브러리
import datetime as dt                                       # 날짜와 시간을 다루는 라이브러리
import json                                                 # JSON 형식의 데이터를 처리하는 라이브러리

import optuna
import pprint

import taipy as tp
from taipy import Config, Orchestrator, Gui

In [3]:
# # Ensure the orchestrator is stopped before modifying the configuration
# tp.Orchestrator().stop()
# Config.configure_job_executions(mode="standalone", max_nb_of_workers=2)

In [4]:
from config.config import scenario_cfg

In [5]:
tp.Orchestrator().stop()
tp.Orchestrator().run()

[2025-08-25 08:12:03.863][Taipy][INFO] Unblocking configuration update...
[2025-08-25 08:12:03.864][Taipy][INFO] Stopping job dispatcher...
[2025-08-25 08:12:03.864][Taipy][INFO] Orchestrator service has been stopped.
[2025-08-25 08:12:03.864][Taipy][INFO] Updating configuration with command-line arguments...
[2025-08-25 08:12:03.866][Taipy][INFO] Managing application's version...
[2025-08-25 08:12:03.872][Taipy][INFO] Development mode: Clean all entities of version 7035f5f3-a7f5-4c93-ba4b-89c7f4ba024c
[2025-08-25 08:12:03.890][Taipy][INFO] Checking application's version...
[2025-08-25 08:12:03.893][Taipy][INFO] Blocking configuration update...
[2025-08-25 08:12:03.894][Taipy][INFO] Starting job dispatcher...
[2025-08-25 08:12:03.896][Taipy][INFO] Orchestrator service has been started.


In [6]:
from algos.algos import *

In [7]:
# 디렉토리 변경
# import os
# import subprocess

# # # 파일이 위치한 디렉토리로 이동
# # # 'c:\\my_project'와 같은 경로를 자신의 파일 경로로 바꿔주세요.
# # os.chdir('/usr01/automl/sparc-malfunction-classification-develop/src/config')
# os.chdir('/usr01/automl/sparc-malfunction-classification-develop/src')


In [8]:
# # 'my_script.py' 파일 실행
# # 이 명령어는 Windows의 경우입니다.
# # MacOS나 Linux에서는 'python my_script.py'로 실행할 수 있습니다.
# subprocess.run(['python', 'config.py'])

# # # 특정 경로를 직접 지정하여 실행
# # # 이 방법은 현재 작업 디렉토리를 바꾸지 않고 파일을 실행할 수 있습니다.
# # subprocess.run(['python', 'C:\\Users\\your_name\\Documents\\my_project\\my_script.py'])

In [9]:
# Config.load(r"/usr01/automl/sparc-malfunction-classification-develop/src/config/config.toml")
# scenario_cfg = Config.scenarios["churn_classification"]

scenario_1 = tp.create_scenario(scenario_cfg)

SCENARIO

In [10]:
# # scenario_1.submit(wait=True, timeout=300)
# scenario_1.submit(wait=True)

In [11]:
# train_dataset = scenario_1.train_dataset.read()
# feature_selection_info = scenario_1.feature_selection_info.read()
# train_parameters = scenario_1.train_parameters_random_forest_optuna.read()

SEQUENCE

TASK

In [12]:
# train_parameters_default = scenario_1.train_parameters_list_default.read()
# pprint.pprint(train_parameters_default)

In [13]:
initial_dataset = scenario_1.initial_dataset.read()

In [14]:
print(initial_dataset.shape)
# initial_dataset

(4536, 1625)


In [15]:
task = scenario_1.preprocess_dataset
# task.submit(wait=True, timeout=50)
task.submit(wait=True)

[2025-08-25 08:12:15.133][Taipy][INFO] job JOB_preprocess_dataset_8621f8bc-5672-403c-914e-d7375fde26c0 is completed.


In [16]:
preprocessed_dataset = scenario_1.preprocessed_dataset.read()

In [17]:
print(preprocessed_dataset.shape)
# preprocessed_dataset

(4536, 1400)


In [18]:
task = scenario_1.create_train_and_test_data
# task.submit(wait=True, timeout=300)
task.submit(wait=True)

[2025-08-25 08:12:21.190][Taipy][INFO] job JOB_create_train_and_test_data_7c43ab74-0f8a-4c74-94ce-3c9057d43f92 is completed.


In [19]:
split_parameter_info = scenario_1.split_parameter_info.read()
# pprint.pprint(split_parameter_info)

In [20]:
train_dataset = scenario_1.train_dataset.read()
print(train_dataset.shape)
test_dataset = scenario_1.test_dataset.read()
print(test_dataset.shape)

(3617, 1400)
(905, 1400)


In [21]:
# train_dataset

In [22]:
# test_dataset

In [23]:
feature_selector = scenario_1.feature_selector.read()
pprint.pprint(feature_selector)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': False,
                    'apply_feature_xicor_filter': False,
                    'apply_target_linear_corr_filter': False,
                    'apply_target_xicor_filter': False,
                    'apply_variance_filter': True,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.05,
                    'target_xicor_threshold': 0.0,
                    'var_threshold': 0.01}}


variance_filter
start

In [24]:
variance_filter = \
{
    "feature_selector_name": "FeatureFilter",
    "filter_methods": {
        "apply_variance_filter": True,
        "var_threshold": 0.00,
        "apply_target_linear_corr_filter": False,
        "target_linear_corr_threshold": 0.5,
        "apply_target_xicor_filter": False,
        "target_xicor_threshold": 0.5,
        "apply_feature_linear_corr_filter": False,
        "feature_linear_corr_threshold": 0.95,
        "apply_feature_xicor_filter": False,
        "feature_xicor_threshold": 0.9
    }
}

In [25]:
scenario_1.feature_selector.write(variance_filter)
feature_selector = scenario_1.feature_selector.read()
pprint.pprint(feature_selector)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': False,
                    'apply_feature_xicor_filter': False,
                    'apply_target_linear_corr_filter': False,
                    'apply_target_xicor_filter': False,
                    'apply_variance_filter': True,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.5,
                    'target_xicor_threshold': 0.5,
                    'var_threshold': 0.0}}


In [26]:
task = scenario_1.select_feature
task.submit(wait=True)

[2025-08-25 08:12:24.718][Taipy][INFO] job JOB_select_feature_32d2e7da-fc02-4898-a4e0-419ae84590ad is completed.


In [27]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
# feature_selection_info

In [28]:
feature_selection_info["final_feature_count"]

1397

In [29]:
# feature_selection_info["final_features"]

In [30]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]

# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
train_dataset_filtered_variance = train_dataset[columns_to_keep]

In [31]:
# train_dataset_filtered_variance

variance_filter
end

In [33]:
scenario_1.train_dataset.write(train_dataset_filtered_variance)

target_linear_corr_filter
start

In [34]:
target_linear_corr_filter = \
{
    "feature_selector_name": "FeatureFilter",
    "filter_methods": {
        "apply_variance_filter": False,
        "var_threshold": 0.01,
        "apply_target_linear_corr_filter": True,
        "target_linear_corr_threshold": 0.00,
        "apply_target_xicor_filter": False,
        "target_xicor_threshold": 0.5,
        "apply_feature_linear_corr_filter": False,
        "feature_linear_corr_threshold": 0.95,
        "apply_feature_xicor_filter": False,
        "feature_xicor_threshold": 0.9
    }
}

In [35]:
scenario_1.feature_selector.write(target_linear_corr_filter)
feature_selector = scenario_1.feature_selector.read()
# pprint.pprint(feature_selector)

In [36]:
task = scenario_1.select_feature
task.submit(wait=True)

[2025-08-25 08:14:31.324][Taipy][INFO] job JOB_select_feature_e8b32176-64b5-42c3-bf40-5813ae464a5f is completed.


In [37]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
# feature_selection_info

In [38]:
feature_selection_info["final_feature_count"]

1397

In [39]:
# feature_selection_info["final_features"]

In [40]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]

# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
target_linear_corr_filter = train_dataset[columns_to_keep]

In [41]:
target_linear_corr_filter

,X,Y,Radius,AC_GAIN_32 of 1103959_69_1133529_cp1_cp1p5_YPP,AC_GAIN_32 of 1103959_69_1133529_cp1_cp1p5_YPP_QPP_RPP,AC_GAIN_32 of 1103959_69_1133529_cp1p5,AC_GAIN_32 of 1103959_69_1133592_QPP,AC_OFFCALRES of 1103959_69_1133529_cp1,AC_OFFCALRES of 1103959_69_1133529_cp1_cp1p5_YPP,AC_OFFCALRES of 1103959_69_1133529_cp1_cp1p5_YPP_QPP_RPP,...,FAILING_BINOUTS(1) of 1103959_69_1133592_QPP_SbPo,FAILING_BINOUTS(1) of 1103959_69_1133592_QPP_SbPoPo,FAILING_BINOUTS(1) of 1103959_69_1133592_QPP_SbSbPoPo,FAILING_BINOUTS(1) of 1103959_69_JPP_GbGbSbPoPo,FAILING_BINOUTS(1) of 1103959_69_JPP_GbGbSbSbPoPo,FAILING_BINOUTS(1) of 1103959_69_JPP_GbPoPo,FAILING_BINOUTS(1) of 1103959_69_JPP_PoPo,FAILING_BINOUTS(1) of 1103959_69_JPP_SbPoPo,band gap dpat_ok for band gap,Pass/Fail
3292,7,54,54.451814,37,37,37,37,-0.938794,0.9645,-0.6113,...,True,False,False,False,False,False,False,False,True,0
3771,-16,39,42.154478,37,37,37,37,-0.938794,-0.3340,-0.6569,...,True,False,False,False,False,False,False,False,True,0
762,-9,38,39.051248,43,43,43,43,-0.745000,-0.7495,-1.6637,...,False,False,False,False,False,False,True,False,False,0
3191,20,58,61.351447,37,37,37,37,-0.938794,-0.1445,-1.4324,...,True,False,False,False,False,False,False,False,True,0
2798,20,37,42.059482,37,37,37,37,-0.938794,0.5464,-0.6471,...,True,False,False,False,False,False,False,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,19,68,70.604532,43,43,43,43,-1.764800,-0.1122,-1.4072,...,False,False,False,False,False,False,True,False,False,0
2847,20,18,26.907248,37,37,37,37,-0.938794,1.4495,1.2634,...,True,False,False,False,False,False,False,False,True,0
2385,28,19,33.837849,43,43,43,43,-1.072100,1.2146,-1.1416,...,False,False,False,False,False,False,True,False,False,0
417,-15,39,41.785165,43,43,43,43,-1.273000,-0.3281,-1.3931,...,False,False,False,False,False,False,False,False,False,0


target_linear_corr_filter
end

In [42]:
scenario_1.train_dataset.write(target_linear_corr_filter)

In [47]:
target_linear_corr_filter_part = target_linear_corr_filter.iloc[:, -5:]
scenario_1.train_dataset.write(target_linear_corr_filter_part)

target_xicor_filter
start

In [48]:
apply_target_xicor_filter = \
{
    "feature_selector_name": "FeatureFilter",
    "filter_methods": {
        "apply_variance_filter": False,
        "var_threshold": 0.01,
        "apply_target_linear_corr_filter": False,
        "target_linear_corr_threshold": 0.01,
        "apply_target_xicor_filter": True,
        "target_xicor_threshold": 0.0,
        "apply_feature_linear_corr_filter": False,
        "feature_linear_corr_threshold": 0.95,
        "apply_feature_xicor_filter": False,
        "feature_xicor_threshold": 0.9
    }
}

In [49]:
scenario_1.feature_selector.write(apply_target_xicor_filter)
feature_selector = scenario_1.feature_selector.read()
# pprint.pprint(feature_selector)

In [51]:
task = scenario_1.select_feature
task.submit(wait=True)

[2025-08-25 08:17:12.918][Taipy][INFO] job JOB_select_feature_490e8641-aeb7-454a-bde6-d88197e39af6 is skipped.


In [52]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
# feature_selection_info

In [53]:
feature_selection_info["final_feature_count"]

4

In [54]:
# feature_selection_info["final_features"]

In [55]:
# feature_selection_info

In [56]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]

# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
target_xicor_filter = train_dataset[columns_to_keep]

In [57]:
# target_xicor_filter

target_xicor_filter
end

feature_linear_corr_filter
start

In [58]:
feature_linear_corr_filter = \
{
    "feature_selector_name": "FeatureFilter",
    "filter_methods": {
        # "apply_variance_filter": False,
        # "var_threshold": 0.01,
        # "apply_target_linear_corr_filter": False,
        # "target_linear_corr_threshold": 0.00,
        # "apply_target_xicor_filter": False,
        # "target_xicor_threshold": 0.5,
        "apply_feature_linear_corr_filter": True,
        "feature_linear_corr_threshold": 0.95,
        # "apply_feature_xicor_filter": False,
        # "feature_xicor_threshold": 0.9
    }
}

In [59]:
# # train_dataset_filtered_variance df의 앞 10개와 맨 뒤 1개 컬럼을 선택
# cols_to_keep = train_dataset_filtered_variance.columns[:100].tolist()
# cols_to_keep.append(train_dataset_filtered_variance.columns[-1])

# # 선택된 컬럼들로 새로운 데이터프레임 생성
# train_dataset_filtered_variance_100 = train_dataset_filtered_variance[cols_to_keep]

# # 결과를 확인하기 위해 새로운 데이터프레임의 컬럼 출력
# print("선택된 컬럼:")
# print(train_dataset_filtered_variance_100.columns)

# scenario_1.train_dataset.write(train_dataset_filtered_variance_100)

In [60]:
scenario_1.train_dataset.write(train_dataset_filtered_variance)

In [61]:
scenario_1.feature_selector.write(feature_linear_corr_filter)
feature_selector = scenario_1.feature_selector.read()
# pprint.pprint(feature_selector)

In [62]:
task = scenario_1.select_feature
task.submit(wait=True)

In [63]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
# feature_selection_info

[2025-08-25 08:17:24.390][Taipy][WARNING] JOB_select_feature_ca451609-8c8f-4518-9e5c-3feb3f61b46d is not in the blocked list anymore.
[2025-08-25 08:17:24.491][Taipy][ERROR]  1 errors occurred during execution of job JOB_select_feature_ca451609-8c8f-4518-9e5c-3feb3f61b46d
[2025-08-25 08:17:24.491][Taipy][ERROR] KeyError: 'var_threshold'



In [64]:
feature_selection_info["final_feature_count"]

4

In [65]:
# feature_selection_info["final_features"]

In [66]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]

# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
feature_linear_corr_filter = train_dataset[columns_to_keep]

In [67]:
feature_linear_corr_filter

,FAILING_BINOUTS(1) of 1103959_69_JPP_GbPoPo,FAILING_BINOUTS(1) of 1103959_69_JPP_PoPo,FAILING_BINOUTS(1) of 1103959_69_JPP_SbPoPo,band gap dpat_ok for band gap,Pass/Fail
3292,False,False,False,True,0
3771,False,False,False,True,0
762,False,True,False,False,0
3191,False,False,False,True,0
2798,False,False,False,True,0
...,...,...,...,...,...
151,False,True,False,False,0
2847,False,False,False,True,0
2385,False,True,False,False,0
417,False,False,False,False,0


feature_linear_corr_filter
end

feature_xicor_filter
start

In [68]:
feature_xicor_filter = \
{
    "feature_selector_name": "FeatureFilter",
    "filter_methods": {
        "apply_variance_filter": False,
        "var_threshold": 0.01,
        "apply_target_linear_corr_filter": False,
        "target_linear_corr_threshold": 0.00,
        "apply_target_xicor_filter": False,
        "target_xicor_threshold": 0.5,
        "apply_feature_linear_corr_filter": False,
        "feature_linear_corr_threshold": 0.95,
        "apply_feature_xicor_filter": True,
        "feature_xicor_threshold": 0.9
    }
}

In [69]:
# scenario_1.train_dataset.write(train_dataset_filtered_variance)

In [70]:
# train_dataset_filtered_variance df의 앞 10개와 맨 뒤 1개 컬럼을 선택
cols_to_keep = train_dataset_filtered_variance.columns[:5].tolist()
cols_to_keep.append(train_dataset_filtered_variance.columns[-1])

# 선택된 컬럼들로 새로운 데이터프레임 생성
train_dataset_filtered_variance_part = train_dataset_filtered_variance[cols_to_keep]

# 결과를 확인하기 위해 새로운 데이터프레임의 컬럼 출력
print("선택된 컬럼:")
print(train_dataset_filtered_variance_part.columns)

scenario_1.train_dataset.write(train_dataset_filtered_variance_part)

선택된 컬럼:
Index(['X', 'Y', 'Radius', 'AC_GAIN_32 of 1103959_69_1133529_cp1_cp1p5_YPP',
       'AC_GAIN_32 of 1103959_69_1133529_cp1_cp1p5_YPP_QPP_RPP', 'Pass/Fail'],
      dtype='object')


In [71]:
scenario_1.feature_selector.write(feature_xicor_filter)
feature_selector = scenario_1.feature_selector.read()
pprint.pprint(feature_selector)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': False,
                    'apply_feature_xicor_filter': True,
                    'apply_target_linear_corr_filter': False,
                    'apply_target_xicor_filter': False,
                    'apply_variance_filter': False,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.0,
                    'target_xicor_threshold': 0.5,
                    'var_threshold': 0.01}}


In [72]:
task = scenario_1.select_feature
task.submit(wait=True)

[2025-08-25 08:19:01.904][Taipy][INFO] job JOB_select_feature_29b1e1d5-bc48-4141-bae9-c438c8148104 is completed.


In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
# feature_selection_info

In [ ]:
feature_selection_info["final_feature_count"]

In [ ]:
# feature_selection_info["final_features"]

In [ ]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]

# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
feature_linear_corr_filter = train_dataset[columns_to_keep]

In [ ]:
feature_linear_corr_filter

feature_xicor_filter
end

In [ ]:
scenario_1.train_dataset.write(feature_linear_corr_filter)

In [ ]:
#@@@@

In [ ]:
task = scenario_1.select_feature
task.submit(wait=True)

In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()

In [ ]:
feature_selection_info

In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
pprint.pprint(feature_selection_info.get('final_feature_count'))

### TASKS train_model

In [ ]:
# Iterate through all scenarios and models
for scenario in tp.get_scenarios():
    for task_name in scenario.tasks:
        if "train_model" in task_name:
            model = task_name.split("train_model_")[1]
            print(f"Model: {model}")
            task = eval("scenario." + task_name)
            task.submit(wait=True)

In [ ]:
task = scenario_1.train_model_baseline
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_baseline = scenario_1.trained_model_baseline.read()
print(trained_model_baseline)
feature_importance_baseline = scenario_1.feature_importance_baseline.read()
print(feature_importance_baseline.shape)

In [ ]:
# train_parameters_logistic_regression = scenario_1.train_parameters_logistic_regression.read()
# train_parameters_logistic_regression

In [ ]:
task = scenario_1.train_model_logistic_regression
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_logistic_regression = scenario_1.trained_model_logistic_regression.read()
print(trained_model_logistic_regression)
feature_importance_logistic_regression = scenario_1.feature_importance_logistic_regression.read()
print(feature_importance_logistic_regression.shape)

In [ ]:
# train_parameters_info_logistic_regression = scenario_1.train_parameters_info_logistic_regression.read()
# pprint.pprint(train_parameters_info_logistic_regression)

In [ ]:
# trained_model_logistic_regression
# feature_importance_logistic_regression
# train_parameters_info_logistic_regression

테스트 

In [ ]:
train_dataset = scenario_1.train_dataset.read()
# train_dataset

In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()


In [ ]:
# train_parameters_logistic_regression = scenario_1.train_parameters_logistic_regression.read()
# train_parameters = train_parameters_logistic_regression
# train_parameters

In [ ]:
# train_parameters_random_forest = scenario_1.train_parameters_random_forest.read()
# train_parameters = train_parameters_random_forest
# train_parameters

In [ ]:
task = scenario_1.train_model_random_forest
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_random_forest = scenario_1.trained_model_random_forest.read()
print(trained_model_random_forest)
feature_importance_random_forest = scenario_1.feature_importance_random_forest.read()
print(feature_importance_random_forest.shape)

In [ ]:
task = scenario_1.train_model_xgboost
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_xgboost = scenario_1.trained_model_xgboost.read()
print(trained_model_xgboost)
feature_importance_xgboost = scenario_1.feature_importance_xgboost.read()
print(feature_importance_xgboost.shape)

### TASKS predict_the_test_data

In [ ]:
task = scenario_1.predict_the_test_data_baseline
# task.submit(wait=True, timeout=50)
task.submit(wait=True, timeout=1)
forecast_dataset_baseline = scenario_1.forecast_dataset_baseline.read()
print(forecast_dataset_baseline)
shap_values_baseline = scenario_1.shap_values_baseline.read()


In [ ]:
# forecast_dataset_baseline

In [ ]:
# shap_values_baseline이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_baseline을 사용하세요.
# shap_values_baseline = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_baseline[0], type(None)):
    print(type(shap_values_baseline[0]))
    print(shap_values_baseline[1].shape)
else:
    shap.summary_plot(shap_values_baseline[0], shap_values_baseline[1])

In [ ]:
task = scenario_1.predict_the_test_data_logistic_regression
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
forecast_dataset_logistic_regression = scenario_1.forecast_dataset_logistic_regression.read()

In [ ]:
print(forecast_dataset_logistic_regression)
shap_values_logistic_regression = scenario_1.shap_values_logistic_regression.read()


In [ ]:

# shap_values_logistic_regression이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_logistic_regression을 사용하세요.
# shap_values_logistic_regression = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_logistic_regression[0], type(None)):
    print(type(shap_values_logistic_regression[0]))
    print(shap_values_logistic_regression[1].shape)
else:
    shap.summary_plot(shap_values_logistic_regression[0], shap_values_logistic_regression[1])

In [ ]:
task = scenario_1.predict_the_test_data_random_forest
task.submit(wait=True, timeout=50)
forecast_dataset_random_forest = scenario_1.forecast_dataset_random_forest.read()
print(forecast_dataset_random_forest)
shap_values_random_forest = scenario_1.shap_values_random_forest.read()

# shap_values_random_forest이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_random_forest을 사용하세요.
# shap_values_random_forest = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_random_forest[0], type(None)):
    print(type(shap_values_random_forest[0]))
    print(shap_values_random_forest[1].shape)
else:
    shap.summary_plot(shap_values_random_forest[0], shap_values_random_forest[1])

In [ ]:
task = scenario_1.predict_the_test_data_xgboost
task.submit(wait=True, timeout=50)
forecast_dataset_xgboost = scenario_1.forecast_dataset_xgboost.read()
print(forecast_dataset_xgboost)
shap_values_xgboost = scenario_1.shap_values_xgboost.read()

# shap_values_xgboost이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_xgboost을 사용하세요.
# shap_values_xgboost = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_xgboost[0], type(None)):
    print(type(shap_values_xgboost[0]))
    print(shap_values_xgboost[1].shape)
else:
    shap.summary_plot(shap_values_xgboost[0], shap_values_xgboost[1])

### TASKS find_best_threshold

In [ ]:
task = scenario_1.find_best_threshold_baseline
task.submit(wait=True, timeout=50)
train_dataset_proba_baseline = scenario_1.train_dataset_proba_baseline.read()
print(train_dataset_proba_baseline.shape)
threshold_baseline = scenario_1.threshold_baseline.read()
print(threshold_baseline)

In [ ]:
task = scenario_1.find_best_threshold_logistic_regression
task.submit(wait=True, timeout=50)
train_dataset_proba_logistic_regression = scenario_1.train_dataset_proba_logistic_regression.read()
print(train_dataset_proba_logistic_regression.shape)
threshold_logistic_regression = scenario_1.threshold_logistic_regression.read()
print(threshold_logistic_regression)

In [ ]:
task = scenario_1.find_best_threshold_random_forest
task.submit(wait=True, timeout=50)
train_dataset_proba_random_forest = scenario_1.train_dataset_proba_random_forest.read()
print(train_dataset_proba_random_forest.shape)
threshold_random_forest = scenario_1.threshold_random_forest.read()
print(threshold_random_forest)

In [ ]:
task = scenario_1.find_best_threshold_xgboost
task.submit(wait=True, timeout=50)
train_dataset_proba_xgboost = scenario_1.train_dataset_proba_xgboost.read()
print(train_dataset_proba_xgboost.shape)
threshold_xgboost = scenario_1.threshold_xgboost.read()
print(threshold_xgboost)

### TASKS roc

In [ ]:
task = scenario_1.task_roc_baseline
task.submit(wait=True, timeout=50)
roc_data_baseline = scenario_1.roc_data_baseline.read()
print(roc_data_baseline.shape)
auc_score_baseline = scenario_1.auc_score_baseline.read()
print(auc_score_baseline)

In [ ]:
task = scenario_1.task_roc_logistic_regression
task.submit(wait=True, timeout=50)
roc_data_logistic_regression = scenario_1.roc_data_logistic_regression.read()
print(roc_data_logistic_regression.shape)
auc_score_logistic_regression = scenario_1.auc_score_logistic_regression.read()
print(auc_score_logistic_regression)

In [ ]:
task = scenario_1.task_roc_random_forest
task.submit(wait=True, timeout=50)
roc_data_random_forest = scenario_1.roc_data_random_forest.read()
print(roc_data_random_forest.shape)
auc_score_random_forest = scenario_1.auc_score_random_forest.read()
print(auc_score_random_forest)

In [ ]:
task = scenario_1.task_roc_xgboost
task.submit(wait=True, timeout=50)
roc_data_xgboost = scenario_1.roc_data_xgboost.read()
print(roc_data_xgboost.shape)
auc_score_xgboost = scenario_1.auc_score_xgboost.read()
print(auc_score_xgboost)

### TASKS create_metrics_on_train

In [ ]:
task = scenario_1.create_metrics_on_train_baseline
task.submit(wait=True, timeout=50)
train_dataset_metrics_baseline = scenario_1.train_dataset_metrics_baseline.read()
print(train_dataset_metrics_baseline.shape)

In [ ]:
task = scenario_1.create_metrics_on_train_logistic_regression
task.submit(wait=True, timeout=50)
train_dataset_metrics_logistic_regression = scenario_1.train_dataset_metrics_logistic_regression.read()
print(train_dataset_metrics_logistic_regression.shape)

In [ ]:
task = scenario_1.create_metrics_on_train_random_forest
task.submit(wait=True, timeout=50)
train_dataset_metrics_random_forest = scenario_1.train_dataset_metrics_random_forest.read()
print(train_dataset_metrics_random_forest.shape)


In [ ]:
task = scenario_1.create_metrics_on_train_xgboost
task.submit(wait=True, timeout=50)
train_dataset_metrics_xgboost = scenario_1.train_dataset_metrics_xgboost.read()
print(train_dataset_metrics_xgboost.shape)

### TASKS task_create_metrics

In [ ]:
task = scenario_1.task_create_metrics_baseline
task.submit(wait=True, timeout=1)
metrics_baseline = scenario_1.metrics_baseline.read()
for key, value in metrics_baseline.items(): print(f'{key}: {value}')

In [ ]:
task = scenario_1.task_create_metrics_logistic_regression
task.submit(wait=True, timeout=1)
metrics_logistic_regression = scenario_1.metrics_logistic_regression.read()
for key, value in metrics_logistic_regression.items(): print(f'{key}: {value}')

In [ ]:
task = scenario_1.task_create_metrics_random_forest
task.submit(wait=True, timeout=1)
metrics_random_forest = scenario_1.metrics_random_forest.read()
for key, value in metrics_random_forest.items(): print(f'{key}: {value}')

In [ ]:
task = scenario_1.task_create_metrics_xgboost
task.submit(wait=True, timeout=1)
metrics_xgboost = scenario_1.metrics_xgboost.read()
for key, value in metrics_xgboost.items(): print(f'{key}: {value}')

### TASKS task_create_results

In [ ]:
task = scenario_1.task_create_results_baseline
task.submit(wait=True, timeout=1)
results_baseline = scenario_1.results_baseline.read()
print(results_baseline.shape)

In [ ]:
task = scenario_1.task_create_results_logistic_regression
task.submit(wait=True, timeout=1)
results_logistic_regression = scenario_1.results_logistic_regression.read()
print(results_logistic_regression.shape)

In [ ]:
results_logistic_regression

In [ ]:
task = scenario_1.task_create_results_random_forest
task.submit(wait=True, timeout=1)
results_random_forest = scenario_1.results_random_forest.read()
print(results_random_forest.shape)

In [ ]:
task = scenario_1.task_create_results_xgboost
task.submit(wait=True, timeout=1)
results_xgboost = scenario_1.results_xgboost.read()
print(results_xgboost.shape)

In [ ]:
# scenarios = tp.get_entities_by_config_id(tp.get_scenarios()[0])
# print(scenarios)
# scenario = scenarios[0]

In [ ]:
# print_object_attributes(scenario)

In [ ]:
# print(scenario.get_inputs())
# print(scenario.get_outputs())

In [ ]:
# input_nodes = scenario.get_inputs()
# output_nodes = scenario.get_outputs()

In [ ]:
# input_node_1 = list(input_nodes)[0]
# output_node_1 = list(output_nodes)[0]
# # output_node_1 = list(output_node_1)[0].read()


In [ ]:
# list(output_nodes)[0]
# print_object_attributes(list(output_nodes)[0])

In [ ]:
# for o_n in list(output_nodes):
#     # print(o_n._config_id )
#     if o_n._config_id.startswith('results_'): 
#         print(o_n._config_id)

# # o_n_1.read()

In [ ]:
# print_object_attributes(input_node_1)

In [ ]:
# print_object_attributes(output_node_1)

In [ ]:
# for i in list(outputs):
#     a=i.read()
#     print(type(a))
    

In [ ]:
# a = tp.get_entities_by_config_id(list(scenario_1.data_nodes)[0])

In [ ]:
# a[0].read()

In [ ]:
# for i in scenario_1.get_outputs(): print(i)

In [ ]:
# list(scenario_1.data_nodes.keys())

In [ ]:
# print([(s.name, s.get_outputs().read()) for s in tp.get_scenarios()])